# Aufgabe-F-FilmDialoge

## Allgemeines

Eine allgemeine Beschreibung der Laboraufgaben inklusive des Vorgehens, den Bewertungsrichtlinien und der Abgabe finden Sie  <a href="ML-allgemein.ipynb">hier</a>

## Datenquelle


* Laden Sie ihre Daten von http://141.72.190.207/ml_lab/F_dialoge herunter
    * Die Daten sind geschützt. 
        * Sie müssen evtl. in einem Netzwerk der DHBW (z.B. WLAN, VPN, ...) angemeldet sein. 
        * Sie können sich auf der Webseite mit dem Benutzernamen dhbw und dem Zugangsnamen "ml_LaB_4$" anmelden. 
* Die Daten sind in einem anwendungsspezifischen Format gespeichert.
    * Sie finden evtl. Informationen über die Daten in einer "README" Datei. 
    * Finden Sie keine solche Datei sind die Daten selbst erklärend. 
    
    



## Aufgabe

Die Daten-Sammlung 
* besteht aus Dialogen aus verschiedensten Filmen
* ist in der Readme Datei beschrieben

Erstellen Sie aus den einen Chatbot, der auf eine Frage mit einer Antwort im "Filmjargon" antwortet! 
* Verwenden Sie tiefe Neuronale Netze zu Erstellen des Chatbots! 
* Passen Sie den Chatbot so an, dass er für unterschiedliche Film-Genres unterschiedlich antwortet! 


# Lösung

Die Lösung der Aufgabe besteht aus mehreren Teilschritten, welche im Folgenden kurz genannt werden:

* Daten einlesen
* Daten vorverarbeiten
* Model erstellen
* Model trainieren
* Model abspeichern
* Model ausführen


# Abhängigkeiten des Projekts

* Tensorflow 2.8.2

In [ ]:
import tensorflow as tf
import random 
import re #re = regular expressions
import os
import numpy as np 
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

# Eigene Abhängigkeiten des Projekts

* 

In [ ]:
import src.data_helper as dh

## Konstanten

In [ ]:
num_examples = 10000
test_size = 0.2

# Daten einlesen

Hier werden zuerst die rohen Daten eingelesen und innerhalb der Hilfsfunktion "readDataToLines" mit des "newline" Zeichen getrennt. Die Daten werden zwei Listen gespeichert, welche dann zurückgegeben werden.
Verwendet wurden folgende Funktionen:

* "open" um die Datei zu öffnen
* "read" um die Datei zu lesen
* "split" um die Daten anhand des "newline" Zeichen zu trennen und in einer Liste zu speichern

In [ ]:
# read the data from the files
movie_lines, movie_conversations = dh.readDataToLines("data/unzipped/movie_lines.txt", "data/unzipped/movie_conversations.txt")

# Daten vorverarbeiten

In [ ]:
# extract and mix conversations to a list
conversations_list = dh.readConversationsToList(movie_conversations)
# random.shuffle(conversations_list) #todo maybe add this later 

# create a dictionary that maps each line id to the corresponding line
id2line = dh.readLinesToDict(movie_lines)

# remove all unnecessary characters from the lines and replace short forms with the full words
id2line = dh.cleanLines(id2line)

# split the conversations into requests and responses, each answer is used as a request for the next answer
requests, responses = dh.splitConversationsToRequestAndResponse(conversations_list, id2line)

print(f"Länge Fragen: {len(requests)}")
print(f"Länge Antworten: {len(responses)}")

# delete temporary variables
del(movie_lines, movie_conversations, conversations_list, id2line)

### Parameter wählen

In [ ]:
#os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# get the length of each line of the requests and save the occurrences in a dictionary
lengths_request = {}
for sentence in requests:
    length = len(sentence)
    if length in lengths_request:
        lengths_request[length] += 1
    else:
        lengths_request[length] = 1

# get the length of each line of the responses and save the occurrences in a dictionary
lengths_response = {}
for sentence in responses:
    length = len(sentence)
    if length in lengths_response:
        lengths_response[length] += 1
    else:
        lengths_response[length] = 1

# plot the occurrences of the lengths of the requests and responses in the same plot while only showind x values up to 200
plt.bar(lengths_request.keys(), lengths_request.values(), label="request")
plt.bar(lengths_response.keys(), lengths_response.values(), label="response")
plt.xlim(0, 400)
plt.legend()
plt.show()

In [ ]:
# based on the result set the parameters
max_wordcount_in_sentence = 30

# delete requests/responses with length > max_wordcount_in_sentence
requests, responses = dh.removeLongSequences(requests, responses, 1, max_wordcount_in_sentence)

print(f"Länge Fragen: {len(requests)}")
print(f"Länge Antworten: {len(responses)}")


In [ ]:
# limit the size of the dataset
requests = requests[:num_examples]
responses = responses[:num_examples]

# encapsule the requests and responses with the tokens <S> (Start) and <E> (End)
requests = dh.encapsuleWithTokens(requests, '<S>', '<E>')
responses = dh.encapsuleWithTokens(responses, '<S>', '<E>')#todo muss das so?

# get a dictionary of all unique words with their frequency
word2count = dh.getWord2Count(requests, responses)

# filter out words with a certain frequency
min_wordFrequency = 1
word2count = {k:v for k,v in word2count.items() if v > min_wordFrequency}

Wie man in der obigen Grafik erkennt, sind ab einer Sequenzlänge von 200 nur noch wenige Daten vorhanden. Da die Sequenzlänge maßgeblich auch für den Speicherbedarf in weiteren Schritten ist, wird diese hier auf 200 begrenzt.

In [ ]:
oov_token = '<UNK>'
maxlen = len(word2count) + 1
del(word2count)


tokenizer = Tokenizer(oov_token=oov_token, filters='', lower=False)
tokenizer.fit_on_texts(requests)
tokenizer.fit_on_texts(responses)

vocab_size = len(tokenizer.word_index) + 1


In [ ]:
tokenized_requests = tokenizer.texts_to_sequences(requests)
tokenized_responses = tokenizer.texts_to_sequences(responses)

max_sen_length = dh.get_maximum_sentence_length(tokenized_requests, tokenized_responses)


pad_type = 'post'

input_encoder = pad_sequences(tokenized_requests, padding=pad_type, maxlen=max_sen_length)
input_decoder = pad_sequences(tokenized_responses, padding=pad_type, maxlen=max_sen_length)


In [ ]:
decoder_output = [i[1:] for i in tokenized_responses]
decoder_output = pad_sequences(decoder_output, maxlen=max_sen_length, padding='post')

# input_encoder_train, input_encoder_val, input_decoder_train, input_decoder_val = train_test_split(input_tensor, target_tensor, test_size=test_size)
# todo?

decoder_output = tf.keras.utils.to_categorical(decoder_output, num_classes=vocab_size)

# Decoder Output erstellen (removed start, because don't want this to be generated)
# output_decoder_train = [i[1:] for i in input_decoder_train]
# output_decoder_val = [i[1:] for i in input_decoder_val]

#Dimension erhöhen
#output_decoder_train = pad_sequences(output_decoder_train, padding=pad_type, maxlen=max_sen_length)
#output_decoder_val = pad_sequences(output_decoder_val, padding=pad_type, maxlen=max_sen_length)

# convert the decoder input to a one-hot encoded vector which the model can use
#output_decoder_train = tf.keras.utils.to_categorical(output_decoder_train, num_classes=vocab_size) # using float32 even though it's memory intensive because later steps need it to be float32
#output_decoder_val = tf.keras.utils.to_categorical(output_decoder_val, num_classes=vocab_size, dtype="float32")

# Model erstellen

## Hyperparameter

In [ ]:
outputDimension = 50 
lstm_units = 256
max_sentence_length = max_sen_length

## Encoder
- Input
- Embedding
- LSTM

In [ ]:
# input tensor for the encoder, shape of each vector is determined by max_length which was also used to pad the data
inputEncoderTensor = tf.keras.Input(shape=(None, ))

# embedding layer of the encoder, the input is the input tensor, the output is the embedding tensor
encoderEmbedding = tf.keras.layers.Embedding(vocab_size, outputDimension, mask_zero=True)(inputEncoderTensor)

# LSTM layer of the encoder, the input is the embedding tensor, the output is the output tensor and the hidden state of the encoder
encoderLSTM, encoderHiddenState, encoderCellState = tf.keras.layers.LSTM(outputDimension, return_state = True)(encoderEmbedding)

## Decoder
- Input
- Embedding
- LSTM
- Dense

In [ ]:
# input tensor for the decoder, shape of each vector is determined by max_length which was also used to pad the data
inputDecoderTensor = tf.keras.Input(shape=(None, ))

# embedding layer of the decoder, the input is the input tensor, the output is the embedding tensor
decoderEmbedding = tf.keras.layers.Embedding(vocab_size, outputDimension, mask_zero=True)(inputDecoderTensor)

# LSTM layer of the decoder, the input is the embedding tensor and the state of the previous lstm layer, the output is the output tensor and the hidden state of the decoder
decoderLSTM = tf.keras.layers.LSTM(outputDimension, return_state = True, return_sequences=True)
decoder_outputs, _, _ = decoderLSTM(decoderEmbedding, initial_state = [encoderHiddenState, encoderCellState])

# dense layer of the decoder, the input is the output tensor of the lstm layer, the output is the output tensor of the dense layer
# the dense layer has the same number of units as the number of words in the dictionary because the output of the dense layer is a vector with a probability for each word in the dictionary
decoderDense = tf.keras.layers.Dense(vocab_size, activation = "softmax")
output = decoderDense(decoder_outputs)

## Summary

In [ ]:
# Define the model 
model = tf.keras.models.Model([inputEncoderTensor, inputDecoderTensor], output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

model.summary()

# Trainieren des Models

In [ ]:
# generate an early stopping callback to stop training when the validation loss stops improving so that the model doesn't overfit
# waits 3 epochs before stopping
# use val_loss as the metric because categorical_crossentropy calculates the difference between the predicted and actual values and by monitoring wether or not the loss would be decreasing or increasing we can see if the model is improving or not
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# Train the model on the training data and evaluate it on the validation data
model.fit([input_encoder, input_decoder], decoder_output, epochs=10, batch_size=50, callbacks=[earlyStopping])

In [ ]:
# Evaluate the model on the validation set and store the results

# loss = model.evaluate([input_encoder_val, input_decoder_val], output_decoder_val)
# print('Test loss:', loss)

# Model abspeichern

In [ ]:
# Modell speichern
dh.save_model(model, 'model.h5')

# # Gewichte speichern
# for layer in model.layers:
#     weights = layer.get_weights()
#     if weights != []:
#         np.savez(f'models/{layer.name}.npz', weights, dtype=object)

Model verwenden

In [ ]:
m = load_model("models/model.h5")

m.save_weights("models/weights.h5")

In [ ]:
# two inputs for the state vectors returned by encoder
dec_state_input_h = Input(shape=(outputDimension,))
dec_state_input_c = Input(shape=(outputDimension,))
dec_states_inputs = [dec_state_input_h, dec_state_input_c]
# these state vectors are used as an initial state 
# for LSTM layer in the inference decoder
# third input is the Embedding layer as explained above
dec_outputs, state_h, state_c = decoderLSTM(decoderEmbedding,
                                initial_state=dec_states_inputs)
dec_states = [state_h, state_c]
# Dense layer is used to return OHE predicted word
dec_outputs = decoderDense(dec_outputs)
dec_model = Model(
    inputs=[inputDecoderTensor] + dec_states_inputs,
    outputs=[dec_outputs] + dec_states, name="decoder_model")

# single encoder input is a question, represented as a sequence 
# of integers padded with zeros
enc_model = Model(inputs=inputEncoderTensor, outputs=[encoderHiddenState, encoderCellState], name="encoder_model")

print(enc_model.summary())
print(dec_model.summary())

# load saved weights to encoder and decoder models
enc_model.load_weights('models/weights.h5', by_name=True)
dec_model.load_weights('models/weights.h5', by_name=True)

In [ ]:
def str_to_tokens(sentence: str):
    # convert input string to lowercase, 
    # then split it by whitespaces
    words = sentence.lower().split()
    # and then convert to a sequence 
    # of integers padded with zeros
    tokens_list = list()
    for current_word in words:
        result = tokenizer.word_index.get(current_word, '')
        if result != '':
            tokens_list.append(result)
    return pad_sequences([tokens_list],
                         maxlen=max_sen_length,
                         padding='post')

In [ ]:

# encode the input sequence into state vectors
states_values = enc_model.predict(
    str_to_tokens("How are you?"))
# start with a target sequence of size 1 - word 'start'   
empty_target_seq = np.zeros((1, 1))
empty_target_seq[0, 0] = tokenizer.word_index['<S>']
stop_condition = False
decoded_translation = ''
while not stop_condition:
    # feed the state vectors and 1-word target sequence 
    # to the decoder to produce predictions for the next word
    dec_outputs, h, c = dec_model.predict([empty_target_seq] 
                                            + states_values)   
    print(dec_outputs)
                  # sample the next word using these predictions
    sampled_word_index = np.argmax(dec_outputs[0, -1, :])
    sampled_word = None
    # append the sampled word to the target sequence
    for word, index in tokenizer.word_index.items():
        if sampled_word_index == index:
            if word != '<E>':
                decoded_translation += ' {}'.format(word)
            sampled_word = word
    # repeat until we generate the end-of-sequence word 'end' 
    # or we hit the length of answer limit
    if sampled_word == '<E>' \
            or len(decoded_translation.split()) \
            > max_sen_length:
        stop_condition = True
    # prepare next iteration
    empty_target_seq = np.zeros((1, 1))
    empty_target_seq[0, 0] = sampled_word_index
    states_values = [h, c]    
    print(decoded_translation)